In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable

open_api_key = os.getenv('OPENAI_API_KEY')

# Initialize OpenAI client

openai_client = OpenAI(api_key = open_api_key)



In [91]:
# 3. Apply LLM definition of attribute values. 
import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import variable_extractor_llm



In [92]:
import importlib
import modules.pdf_extraction
importlib.reload(modules.pdf_extraction)
from modules.pdf_extraction import extract_text_from_pages, select_pdf_file



In [93]:
import multiprocessing

num_cpus = multiprocessing.cpu_count()
print(f"Number of CPUs: {num_cpus}")


Number of CPUs: 16


In [94]:
pdf_path = select_pdf_file()

2025-02-11 00:11:19,741 - INFO - Opening file selection dialog.
2025-02-11 00:11:21,504 - INFO - Selected PDF file: C:/Users/derri/Desktop/page0027.pdf


In [96]:
# 15
page_no = 0
extracted_text = extract_text_from_pages(pdf_path, pages=page_no)


In [97]:
import pymupdf 
## 111
import base64
import requests

import importlib
import modules.llm
import modules.pdf_extraction
importlib.reload(modules.llm)
importlib.reload(modules.pdf_extraction)

from modules.llm import pattern_description_llm, table_extraction
from modules.pdf_extraction import get_page_pixel_data


base64_image = get_page_pixel_data(pdf_path=pdf_path, page_no=page_no, 
                    dpi = 500, image_type = 'png')

# perhaps let it run twice to make super sure. if the numbers do not match run again and pick the max. If it still does not work then just pick one
pattern_desc_from_image = pattern_description_llm(text_input = extracted_text, model='gpt-4o',  base64_image = base64_image, open_api_key=open_api_key)

pattern_desc_from_image

# Function to process each row in parallel
# open ai does not seem to have structed output for this image input version.
# Ill pass it as text to the next llm# 111

'1. Number of Tables on the Page: 6\n\n2. Table Headers: Metric Male 24° Cone Seat, S Series, DIN 3865 || Metric Female 24° Cone Seat, S Series, DIN 3865 || 90° Metric Female 24° Cone Seat, S Series, DIN 3865 || Standpipe, S Series || 45° Metric Female 24° Cone Seat, S Series, DIN 3865 || 90° Standpipe, S Series\n\n3. Rules for Parsing Tables and Headers: \n   - Identify table headers by recognizing bold or differentiated text followed by structured content.\n   - Look for sequences where text is followed by columns with repeated patterns (e.g., Thread, To Suit, Hose ID, Size).\n   - Use contextual key phrases like "Cone Seat," "Standpipe," and angle indicators (e.g., 90°, 45°) to distinguish table groups.\n   - Text structured in consistent columns over multiple lines often indicates a table.'

In [98]:
# Extract the respective variable outputs


import re

text = pattern_desc_from_image
# 1) Extract the Number of Tables
match_num_tables = re.search(r'Number of Tables on the Page:\s*(\d+)', text)
if match_num_tables:
    num_tables = int(match_num_tables.group(1))
else:
    num_tables = None  # or handle error

# 2) Extract the Table Headers
#    We'll grab everything after "Table Headers:" until the line "3. Rules" begins
match_headers = re.search(r'Table Headers:\s*(.*?)\s*\n\s*3\.', text, re.DOTALL)
if match_headers:
    headers_text = match_headers.group(1)
    # Split on '||' to get individual headers
    table_headers = [h.strip() for h in headers_text.split('||')]
else:
    table_headers = []

# 3) Extract the Rules for Parsing Tables and Headers
#    We'll capture everything after "3. Rules for Parsing Tables and Headers:" until the end
match_rules = re.search(r'3\. Rules for Parsing Tables and Headers:\s*(.*)', text, re.DOTALL)
if match_rules:
    parsing_rules = match_rules.group(1).strip()
else:
    parsing_rules = ""

# # Print or use these variables as needed
# print("Number of Tables:", num_tables)
# print("Table Headers:", table_headers)
# print("Rules for Parsing Tables and Headers:\n", parsing_rules)


In [99]:
# Numerical columns as an additional feature

user_text='Extract all the data from the table'

import modules.llm
importlib.reload(modules.llm)
from modules.llm import vision_column_llm_parser
# use user input to determine table of interst
# variable_col_data = vision_column_llm_parser(user_text='Extract all data from the table i also want a column which tells me when the bend radius is above 1000',
#                                     table_to_target= table_headers[0],
#                                     base64_image=base64_image,
#                                     open_api_key=open_api_key)

variable_col_data = vision_column_llm_parser(user_text=user_text,
                                   text_input= extracted_text,
                                    table_to_target= headers_text,
                                    base64_image=base64_image,
                                    open_api_key=open_api_key)
variable_col_data

'``````````````````````\nindex: [0], table_header or descriptor: [Metric Male 24° Cone Seat, S Series, DIN 3865], column_names: [0309, thread, to_suit_hose_id, size]\nindex: [1], table_header or descriptor: [Metric Female 24° Cone Seat, S Series, DIN 3865], column_names: [0309, thread, to_suit_hose_id, size]\nindex: [2], table_header or descriptor: [90° Metric Female 24° Cone Seat, S Series, DIN 3865], column_names: [0309, thread, to_suit_hose_id, size]\nindex: [3], table_header or descriptor: [Standpipe, S Series], column_names: [0309, od_mm, to_suit_hose_id]\nindex: [4], table_header or descriptor: [45° Metric Female 24° Cone Seat, S Series, DIN 3865], column_names: [0309, thread, to_suit_hose_id, size]\nindex: [5], table_header or descriptor: [90° Standpipe, S Series], column_names: [0309, od_mm, to_suit_hose_id]\n``````````````````````'

In [100]:
# import re

# # Your text
# text =variable_col_data

# pattern = r'index:\s*\[(\d+)\].*?column_names:\s*\[(.*?)\]'
# matches = re.findall(pattern, text)

# results = []
# for index_str, columns_str in matches:
#     index_value = int(index_str)
    
#     # 1) Strip any extra whitespace
#     # 2) Then remove leading/trailing brackets (e.g. "[..." or "...]")
#     columns_str = columns_str.strip().strip("[]")
    
#     # Now split on commas to get a proper list
#     columns_list = [col.strip() for col in columns_str.split(',')]
    
#     # Fetch the correct table_header using index
#     header = table_headers[index_value]
    
#     results.append({
#         "index": index_value,
#         "table_header": header,
#         "column_names": columns_list
#     })




In [101]:
#type(results[0]['column_names'])

In [102]:

text = variable_col_data
pattern = r'index:\s*\[(\d+)\].*?column_names:\s*\[(.*?)\]'

matches = re.findall(pattern, text)

results = []
for index_str, columns_str in matches:
    index_value = int(index_str)
    columns_list = [col.strip() for col in columns_str.split(',')]
    
    # Use the table_headers list to fetch the header based on index
    header = table_headers[index_value]
    
    results.append({
        "index": index_value,
        "table_header": header,
        "column_names": columns_list
    })

In [103]:

# import importlib
# import modules.llm
# importlib.reload(modules.llm)
# from modules.llm import variable_extractor_llm

# var_list =  variable_extractor_llm(user_text= 'return every column in the table',
#                                     text_data=extracted_text, 
#                                     target_table= table_headers[2],
#                                     pattern =parsing_rules,
#                                     openai_client=openai_client, 
#                                     model='gpt-4o') # gpt-4o
# var_list



In [104]:
table_index = 0
table_headers[0]

'Metric Male 24° Cone Seat, S Series, DIN 3865'

In [105]:
input_fields = sorted(set(results[table_index]['column_names'])) # in case of dups
input_fields

['0309', 'size', 'thread', 'to_suit_hose_id']

In [106]:
import modules.llm
importlib.reload(modules.llm)
from modules.llm import vision_llm_parser
# use user input to determine table of interst
variable_col_data = vision_llm_parser(user_text=user_text,
                                   text_input= extracted_text,
                                   columns=input_fields,
                                    table_to_target= table_headers[0],
                                    base64_image=base64_image,
                                    open_api_key=open_api_key)
variable_col_data

'To extract the data from the specified table "Metric Male 24° Cone Seat, S Series, DIN 3865", and return it in the structured format, I\'ll focus on the columns [\'0309\', \'size\', \'thread\', \'to_suit_hose_id\'].\n\nHere is the extracted information:\n\n``````````````````````````\n\n[0309: HMMS03-M16CF |-| HMMS04-M14CF |-| HMMS04-M16CF |-| HMMS04-M18CF |-| HMMS05-M18CF |-| HMMS05-M20CF |-| HMMS06-M18CF |-| HMMS06-M20CF |-| HMMS06-M22CF |-| HMMS06-M24CF |-| HMMS08-M24CF |-| HMMS10-M30CF |-| HMMS12-M30CF |-| HMMS12-M36CF |-| HMMS16-M36CF |-| HMMS16-M42CF |-| HMMS20-M42CF |-| HMMS20-M52CF |-| HMMS24-M52CF],\n\n[thread: M16 x 1.5 |-| M14 x 1.5 |-| M16 x 1.5 |-| M18 x 1.5 |-| M18 x 1.5 |-| M20 x 1.5 |-| M18 x 1.5 |-| M20 x 1.5 |-| M22 x 1.5 |-| M24 x 1.5 |-| M24 x 1.5 |-| M30 x 2 |-| M30 x 2 |-| M36 x 2 |-| M36 x 1.5 |-| M42 x 2 |-| M42 x 2 |-| M52 x 2 |-| M52 x 2],\n\n[to_suit_hose_id: 3/16” |-| 1/4” |-| 1/4” |-| 1/4” |-| 5/16” |-| 5/16” |-| 3/8” |-| 3/8” |-| 3/8” |-| 3/8” |-| 1/2” |-|

In [107]:
import pandas as pd
text = variable_col_data
pattern = r"\[([^\]:]+):([^]]+)\]"
matches = re.findall(pattern, text, flags=re.DOTALL)

data = {}
max_len = 0

# 2) Split each matched value on "|-|"
for key, val in matches:
    items = [item.replace("***", "").strip()  # optionally remove "***"
             for item in val.split("|-|")]
    data[key.strip()] = items
    max_len = max(max_len, len(items))

# 3) Build a DataFrame, ensuring all columns have the same length
df = pd.DataFrame({
    col: values + [None]*(max_len - len(values))  # pad with None if needed
    for col, values in data.items()
})

df



,0309,thread,to_suit_hose_id,size
0,HMMS03-M16CF,M16 x 1.5,3/16”,8S
1,HMMS04-M14CF,M14 x 1.5,1/4”,6S
2,HMMS04-M16CF,M16 x 1.5,1/4”,8S
3,HMMS04-M18CF,M18 x 1.5,1/4”,10S
4,HMMS05-M18CF,M18 x 1.5,5/16”,10S
5,HMMS05-M20CF,M20 x 1.5,5/16”,12S
6,HMMS06-M18CF,M18 x 1.5,3/8”,10S
7,HMMS06-M20CF,M20 x 1.5,3/8”,12S
8,HMMS06-M22CF,M22 x 1.5,3/8”,14S
9,HMMS06-M24CF,M24 x 1.5,3/8”,16S


In [108]:
1/0

ZeroDivisionError: division by zero

In [ ]:
# import re
# import pandas as pd

# text = llm_table_data

# pattern = r"\[(\w+):([^]]+)\]"
# matches = re.findall(pattern, text)

# data = {}
# for col_name, raw_values in matches:
#     values = [v.strip() for v in raw_values.split("|-|")]
#     data[col_name] = values

# # --- Fix or pad mismatched lengths -----------------------------
# max_len = max(len(vals) for vals in data.values())
# for col_name, vals in data.items():
#     if len(vals) < max_len:
#         # Append None (or "" or some placeholder) to match the maximum length
#         vals += [None] * (max_len - len(vals))

# df = pd.DataFrame(data)
# df

In [ ]:
# columsn may be changeed due to llm being overhweled.

In [ ]:
# import re
# import pandas as pd

# text = llm_table_data

# # 1) Regex pattern to capture things of the form:
# #    [column_name: val1 |-| val2 |-| ...]
# pattern = r"\[(\w+):([^]]+)\]"

# # 2) Find all matches; each match is (column_name, "val1 |-| val2 |-| ...")
# matches = re.findall(pattern, text)

# # 3) Build a dictionary { column_name: [list_of_values, ...] }
# data = {}
# for col_name, raw_values in matches:
#     # Remove any extra newlines/spaces and split on "|-|"
#     values = [v.strip() for v in raw_values.split("|-|")]
#     data[col_name] = values

# # 4) Create the DataFrame
# df = pd.DataFrame(data)



In [ ]:
# import importlib
# import modules.llm
# importlib.reload(modules.llm)


# from modules.llm import table_extraction


# table_output = table_extraction(pattern_data = pattern_desc_from_image, text_data =extracted_text, model = "gpt-4o", openai_client=openai_client)
# table_output

In [ ]:
results

[{'index': 0,
  'table_header': 'Box SPM.1, Table 1: Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports',
  'column_names': ['category_in_wgiii',
   'category_description',
   'ghg_emissions_scenarios_sspx-y_in_wgi_and_wgii',
   'rcpy_in_wgi_and_wgii']}]

In [ ]:
results[0]['column_names']

['category_in_wgiii',
 'category_description',
 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii',
 'rcpy_in_wgi_wgii']

In [ ]:
input_fields = list(set(results[0]['column_names']))
input_fields

['category_in_wgiii',
 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii',
 'category_description',
 'rcpy_in_wgi_wgii']

In [ ]:
from pydantic import create_model
import json
import pandas as pd 

import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import llm_parser


import importlib
import modules.llm
importlib.reload(modules.llm)
from modules.llm import variable_extractor_llm


# Could targeting tables then extracting from each one by one and then concating be the best way to go ?
# how do i identify tables

# If user provided then

#  pattern = pattern_output,
# If no user provided then get a default template

# pattern_output = pattern_description_llm(text_input=extracted_text, base64_image = base64_image, open_api_key= open_api_key )


# var_list =  variable_extractor_llm(user_text=text_1, text_data=extracted_text, openai_client=openai_client, model='o3-mini') # gpt-4o
table_index = 0

input_fields = sorted(set(results[table_index]['column_names'])) # in case of dups


# Assume these are the field names coming from the front end
# input_fields = list(set(var_list))

# input_fields.append('table_number')  # To differential tables
# print(input_fields)

#   Build a dictionary defining the field names and their types.
#  I will assume each field required a list of strings.
field_definitions = {field: (list[str], ...) for field in input_fields}

# Dynamically create a new Pydantic model 
dynamic_structured_output_class = create_model('structured_output', **field_definitions)

target_table = results[table_index]['table_header']
print(f"Target table {target_table}")
print(f"Columns {input_fields} ")

completion = llm_parser(user_text = user_text, 
                        target_table=target_table,
                        target_variables= input_fields,
                        openai_client = openai_client,
                        pattern=parsing_rules,
                        text_document=extracted_text,
                        response_format=dynamic_structured_output_class,
                        model='gpt-4o') # o3-mini, gpt-4o

out  = completion.choices[0].message.content

data = json.loads(out)

# To ensure all columns are of the same length
# Determine the maximum length among the arrays
max_len = max(len(lst) for lst in data.values())

# Pad each list to have the same length
for key, lst in data.items():
    if len(lst) < max_len:
        lst.extend([None] * (max_len - len(lst)))

# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df # ensure columsn returnd are ascii clean white spaces

#error handling try and except 
# Perhaps retry when JSON error happens
# Factorise table numbers. Sometimes i can choose 2 with no other tables present

Target table Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports
Columns ['category_description', 'category_in_wgiii', 'ghg_emissions_scenarios_sspx_y_in_wgi_wgii', 'rcpy_in_wgi_wgii'] 


2025-02-10 23:55:19,368 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


,category_description,category_in_wgiii,ghg_emissions_scenarios_sspx_y_in_wgi_wgii,rcpy_in_wgi_wgii
0,limit warming to 1.5°C (>50%) with no or limit...,C1,Very low (SSP1-1.9),RCP2.6
1,return warming to 1.5°C (>50%) after a high ov...,C2,Low (SSP1-2.6),
2,limit warming to 2°C (>67%),C3,,
3,limit warming to 2°C (>50%),C4,,
4,limit warming to 2.5°C (>50%),C5,,
5,limit warming to 3°C (>50%),C6,Intermediate (SSP2-4.5),RCP 4.5
6,limit warming to 4°C (>50%),C7,High (SSP3-7.0),RCP 8.5
7,exceed warming of 4°C (>50%),C8,Very high (SSP5-8.5),


In [ ]:
target_table

'Description and relationship of scenarios and modelled pathways considered across AR6 Working Group reports'

In [ ]:
1/0

ZeroDivisionError: division by zero

In [ ]:
1/0

In [ ]:
!pip install pdfplumber

In [ ]:
import pdfplumber
import pandas as pd

def extract_tables_from_pdf(pdf_path, output_csv=None):
    tables = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            extracted_tables = page.extract_tables()
            for table in extracted_tables:
                tables.append(pd.DataFrame(table))
    
    # Combine tables into one DataFrame if needed
    if tables:
        df = pd.concat(tables, ignore_index=True)
        if output_csv:
            df.to_csv(output_csv, index=False)
        return df
    else:
        return None

# Example usage

output_csv = "output.csv"  # Optional: specify a CSV file to save the extracted table
df = extract_tables_from_pdf(pdf_path=pdf_path, output_csv=output_csv)

if df is not None:
    print(df.head())  # Display first few rows
else:
    print("No tables found in the PDF.")


In [ ]:
from img2table.document import PDF

doc = PDF(src=pdf_path, 
          pages=None,
          detect_rotation=False,
          pdf_text_extraction=True)

In [ ]:
from img2table.ocr import TesseractOCR
from img2table.document import Image
import pytesseract

# Instantiation of OCR
ocr = TesseractOCR(n_threads=1, lang="eng")

# Instantiation of document, either an image or a PDF
# doc = Image(pdf_path)

# Table extraction
extracted_tables = doc.extract_tables(ocr=ocr,
                                      implicit_rows=False,
                                      implicit_columns=False,
                                      borderless_tables=False,
                                      min_confidence=50)

In [ ]:
df.drop_duplicates(subset='id_information')

In [ ]:
extracted_text

In [ ]:
dynamic_structured_output_class

In [ ]:
pdf_path

In [ ]:
print(dynamic_structured_output_class.__annotations__)


In [ ]:
data

In [ ]:
# Create a DataFrame from the dictionary
df = pd.DataFrame(data)

df

In [ ]:
extracted_text_list

In [ ]:
out  = completion.choices[0].message.content
out

In [ ]:
# Parse the JSON string into a dictionary
import json
import pandas as pd 

# Display the DataFrame
print(df)

In [ ]:
prompt_variable_extractor = """ 

 You are a skilled data extraction and parsing specialist with expertise in analysing text documents and extracting structured information, these could be from a varity of sources including PDFs and many others. 
          Your job is to extract data for the provided variable(s) from a text document in line with the user's request and return them in JSON format where each value is associated with each other.  

 
          ''''''''''''''''
          User's request: " I would like to extract the Order No and Model from the pdf document. The order number looks something like this (1051042839) and the model No is an mixture of numbers and letters and looks like this for instance NLOLC1
          I would also like to extract the heading which is usually uppercased and above the order and model number. 
          ''''''''''''''''
           
          Target variable(s): [model_number, order_number]
           
text document: ['\n\n--- Page 3 ---\n\n\tSTEEL ENCLOSED CONTACTORS\nOrder No.\nModel No.\nAC1 Heating & \nGeneral Load\nFluorescent \nMercury\nHalogen\nLED\nNo. of Poles\nDimensions HxWxD\nSTEEL ENCLOSED CONTACTORS\n1051042857\nNLCONM25/4\n25A\n16A\n13A\n25A\n4 + N\n184 x 184 x 149mm\n1051042858\nNLCONM45/4\n40A\n27A\n22A\n40A\n4 + N\n184 x 184 x 149mm\n1051042859\nNLCONM63/3N\n60A\n40A\n32A\n60A\n3 + N\n305 x 285 x 159mm\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tStandard coil voltage: 230V 50/60Hz\n•\tAuxiliary contacts can be fitted\n•\tColour RAL7035 light grey\n•\tKnockouts top and bottom\n•\tAll units are suitable for two wire control i.e. time \nswitch, thermostat, sensor or other remote controls\n•\tSupplied complete with neutral terminals\nNLCONM100/3N\n\tDIRECT ON LINE, REVERSING & \nSTAR DELTA STARTERS\n•\tManufactured in accordance with EN60947-4-1 \n•\tIP55 steel enclosure\n•\tColour RAL7035 light grey\n•\tSurface mounting\n•\tStandard coil voltages: 230V or 400V 50/60Hz\n•\tBuilt in start/stop and reset push button\n•\tHand reset thermal overloads\n•\tOverload to be ordered separately\n•\tStainless steel pozidrive fixing screws for lid\n•\tKnockouts top and bottom\n•\tInternal components DIN rail mounted\nOrder No.\nModel No.\nMax Rating\nControl Voltage\nDimensions H x W x D\nType\nDIRECT ON LINE & REVERSING STARTERS\n1051042849\nNLSTM27C\n7.5kW\n230V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042850\nNLSTM47C\n7.5kW\n400V\n174 x 104 x 134mm\nDirect on Line\n£0.00\n1051042856\nNLRVM45C\n5.5kW\n400V\n184 x 184 x 149mm\nReversing\n£0.00\n1051042851\nNLSTM25CSW\n5.5kW\n230V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042852\nNLSTM45CSW\n5.5kW\n400V\n184 x 184 x 149mm\nDOL With Isolator\n£0.00\n1051042854\nNLSDM415C\n15kW\n400V\n184 x 284 x 149mm\nStar Delta\n£0.00\nNLSTM27C\nStock items delivered FREE anywhere in UK\n15 3\n\n',
 '\n']

Output:

[
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042857",
    "model_number": "NLCONM25/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042858",
    "model_number": "NLCONM45/4"
  },
  {
    "heading": "STEEL ENCLOSED CONTACTORS",
    "order_number": "1051042859",
    "model_number": "NLCONM63/3N"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042849",
    "model_number": "NLSTM27C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042850",
    "model_number": "NLSTM47C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042856",
    "model_number": "NLRVM45C"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042851",
    "model_number": "NLSTM25CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042852",
    "model_number": "NLSTM45CSW"
  },
  {
    "heading": "DIRECT ON LINE, REVERSING & STAR DELTA STARTERS",
    "order_number": "1051042854",
    "model_number": "NLSDM415C"
  }
]


"""
